In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('AirBnB.csv')

In [3]:
columns = ['neighbourhood_group', 'room_type','latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

df = df[columns].fillna(0)
df.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Private room,40.64749,-73.97237,149,1,9,0.21,6,365
1,Manhattan,Entire home/apt,40.75362,-73.98377,225,1,45,0.38,2,355
2,Manhattan,Private room,40.80902,-73.94190,150,3,0,0.00,1,365
3,Brooklyn,Entire home/apt,40.68514,-73.95976,89,1,270,4.64,1,194
4,Manhattan,Entire home/apt,40.79851,-73.94399,80,10,9,0.10,1,0


In [4]:
df['neighbourhood_group'].mode()

0    Manhattan
dtype: object

In [5]:
categorical = ['neighbourhood_group', 'room_type']
df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# Split the data w/ seed 42
np.random.seed(42)
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 1)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 1)

In [8]:
df_train.reset_index(drop = True)
df_val.reset_index(drop = True)
df_test.reset_index(drop = True)

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,queens,private_room,40.74596,-73.87762,60,2,0,0.00,2,66
1,queens,entire_home/apt,40.75883,-73.92160,250,2,0,0.00,1,282
2,brooklyn,private_room,40.69670,-73.97477,80,1,53,2.15,1,0
3,brooklyn,private_room,40.72435,-73.95154,95,1,119,2.57,2,1
4,manhattan,entire_home/apt,40.72754,-73.98555,450,3,12,0.19,2,0
...,...,...,...,...,...,...,...,...,...,...
9774,bronx,private_room,40.85062,-73.90251,40,14,5,0.13,1,322
9775,brooklyn,entire_home/apt,40.70450,-73.94357,150,6,5,2.17,1,40
9776,brooklyn,private_room,40.65100,-73.94886,50,2,263,2.69,2,136
9777,brooklyn,entire_home/apt,40.68040,-73.93183,105,7,53,1.12,2,77


In [9]:
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

In [10]:
del df_train['price']
del df_val['price']
del df_test['price']

In [11]:
# Create the correlation matrix
df_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [12]:
numerical = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
df_corr = df[numerical].corr()

In [13]:
df_corr

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.084788,0.024869,-0.015389,-0.018758,0.019517,-0.010983
longitude,0.084788,1.000000,-0.062747,0.059094,0.138516,-0.114713,0.082731
minimum_nights,0.024869,-0.062747,1.000000,-0.080116,-0.124905,0.127960,0.144303
number_of_reviews,-0.015389,0.059094,-0.080116,1.000000,0.589407,-0.072376,0.172028
reviews_per_month,-0.018758,0.138516,-0.124905,0.589407,1.000000,-0.047312,0.163732
calculated_host_listings_count,0.019517,-0.114713,0.127960,-0.072376,-0.047312,1.000000,0.225701
availability_365,-0.010983,0.082731,0.144303,0.172028,0.163732,0.225701,1.000000


In [44]:
# Create a variable above_average which is 1 if the price is above (or equal to) 152.
above_average = (y_train >= 152).astype(int)
above_average

array([0, 0, 1, ..., 0, 1, 1])

In [15]:
from sklearn.metrics import mutual_info_score

In [45]:
# Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
# Which of these two variables has bigger score?
score = mutual_info_score(above_average, df_train['neighbourhood_group'])
round(score, 2)

0.05

In [46]:
score = mutual_info_score(above_average, df_train['room_type'])
round(score, 2)

0.14

In [18]:
from sklearn.feature_extraction import DictVectorizer

In [47]:
# One-hot encoding
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [33]:
dv = DictVectorizer(sparse=False)

In [34]:
X_train = dv.fit_transform(train_dicts)


In [35]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
from sklearn.linear_model import LogisticRegression

In [36]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)

In [48]:
model.fit(X_train, (y_train >= 152).astype(int))

C:\Users\Luke Luo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [49]:
model.coef_[0].round(3)

array([ 0.003,  0.004, -0.218, -0.089, -0.01 , -0.366,  0.098,  1.197,
       -0.798, -0.134, -0.003, -0.047,  1.655, -1.216, -0.442])

In [50]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.08999742, 0.36203746, 0.61808978, ..., 0.61900055, 0.62550043,
       0.47035374])

In [51]:
result = y_pred >= .5
result.astype(int)

array([0, 0, 1, ..., 1, 1, 0])

In [55]:
full_mean = ((y_val >= 152).astype(int) == result).mean()
full_mean

0.7905716330913182

Question 5

We have 9 features: 7 numerical features and 2 categorical.

Let's find the least useful one using the feature elimination technique.

Train a model with all these features (using the same parameters as in Q4).

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.

For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

neighbourhood_group

room_type

number_of_reviews

reviews_per_month

note: the difference doesn't have to be positive

In [57]:
def find_mean_diff(columns):

    train_dicts = df_train[columns].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    val_dicts = df_val[columns].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    model.fit(X_train, (y_train >= 152).astype(int))

    y_pred = model.predict_proba(X_val)[:, 1]
    result = y_pred >= .5
    result.astype(int)
    mean = ((y_val >= 152).astype(int) == result).mean()
    return(full_mean - mean)

In [58]:
# Exclude neighbourhood_group
no_neighbourhood = ['room_type','latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
print(find_mean_diff(no_neighbourhood))

0.04202883730442786


In [59]:
# Exclude room_type
no_room_type = ['neighbourhood_group','latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
print(find_mean_diff(no_room_type))

0.06452602515594641


C:\Users\Luke Luo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [60]:
# Exclude number_of_reviews
no_no_reviews = ['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'minimum_nights', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
print(find_mean_diff(no_no_reviews))

-0.0006135596686777101


C:\Users\Luke Luo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
# Exclude room_type
no_month_reviews = ['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count', 'availability_365']
print(find_mean_diff(no_month_reviews))

0.0004090397791185474


C:\Users\Luke Luo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


For this question, we'll see how to use a linear regression model from Scikit-Learn

We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.

Fit the Ridge regression model on the training data.

This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]

Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest alpha.

In [64]:
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)
y_test_log = np.log1p(y_test)

In [65]:
from sklearn.linear_model import Ridge

In [67]:
def rmse(y, y_pred):
    se = (y-y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [73]:
for a in [0, 0.01, 0.1, 1, 10]:
    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    val_dicts = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    model = Ridge(alpha=a, random_state=42)
    model.fit(X_train, y_train_log)

    y_pred = model.predict(X_val)
    score = rmse(y_val_log, y_pred)
    print(a)
    print(round(score, 3))

0
0.494
0.01
0.494
0.1
0.494
1
0.494
10
0.495
